## Introduction

### Pré-requis

* Ce notebook permet de peupler l'ontologie relative aux adresses définies dans le fichier lié à la variable `ont_file`.
* Il faut s'assurer que l'environnement Python qui va lancer le script possède les librairies suivantes : `os`, `sys`, `ssl`, `xml`, `rdflib`, `urllib`, `importlib`, `SPARQLWrapper`
* Avant de procéder à l'exécution du script, il faut installer deux logiciels qui devront être tournés durant le processus :
    * [Ontotext Refine](https://www.ontotext.com/products/ontotext-refine/) (ou OntoRefine) qui permet d'automatiser la conversion de données textuelles en un graphe de connaissances. Dans la section des variables globables (voir ci-dessous), deux variables sont liées au logiciel :
        * `ontorefine_cli` : chemin absolu de l'interface en ligne de commande (CLI) d'OntoRefine. Pour trouver ce chemin, il suffit de cliquer sur le bouton `Open CLI directory` présent sur l'interface de lancement du logiciel. Cela vous renvoie vers le dossier contenant la CLI dont le nom est `ontorefine-cli`.
        * `ontorefine_url` : l'URL de l'application web.  Pour trouver cette valeur, il suffit de cliquer sur le bouton `Open Refine web application` présent sur l'interface de lancement du logiciel. La valeur à associer à `ontorefine_url` est l'url à laquelle on enlève `/projects`.
    * [GraphDB](https://graphdb.ontotext.com/) qui permet de stocker et de travailler sur des graphes de connaissance. Une variable est associée au logiciel : `graphdb_url` qui est l'URL de l'application web. Le procédé est similaire à celui de `ontorefine_url` pour trouver sa valeur.

![Interface d'OntoRefine avec les boutons \`Open Refine web application\` et \`Open CLI directory\`](/../../img/interface_ontorefine.png)

### Fichiers du dossier
* `ont_file` est le fichier comprenant l'ontologie des adresses.
* `data_file` : le but du notebook est de peupler l'ontologie. Ce peuplement se fait grâce aux données de `{inserer la source des données}` dont les adresses sont stockées localement dans le fichier csv lié à la variable `data_file`.
* `mapping_file` : Afin de convertir les données présentes dans le fichier précédemment cité, un mapping est nécessaire pour les convertir en données RDF selon les recommandations de l'ontologie. Pour cela, le fichier json lié `mapping_file` existe et est utilisé par OntoRefine.
* `query_file` : À la création des données via OntoRefine, il y a des données dupliquées. Un ensemble de requêtes SPARQL détecte ces doublons et supprimer les données inutiles. Ces requêtes sont dans le fichier `query_file`.

### Présentation du processus
* Le processus est divisé en quatre parties :
    * la première fait des traitements généraux :
        * import des librairies
        * définition des variables et modification de ces dernières si nécessaire (conversion des chemins relatifs en chemins absolus pour éviter les mauvaises surprises par exemple)
        * créer de dossiers s'ils n'existent pas (`out_folder` ou `tmp_folder`).
    * la deuxième est relative à OntoRefine :
        * création d'un projet à partir du fichier lié à `data_file` ;
        * conversion en données RDF grâce au fichier `mapping_file` ;
        * export des données créées dans un graphe stocké dans le fichier `out_ontorefine_file` ;
        * suppression du projet.
    * la troisième est liée à l'import des données créées par OntoRefine dans GraphDB :
        * création d'un répertoire ayant `project_name` comme identifiant. S'il existe déjà, on le vide de tous ses triplets. C'est pourquoi il faut éviter de définir un identifiant qui se réfère à un répertoire déjà existant ;
        * import de l'ontologie présente dans `ont_file` et des données créées par OntoRefine qui sont dans `out_ontorefine_file` ;
        * nettoyage des données grâce aux requêtes présentes dans `query_file`.
    * la dernière partie permet de répondre aux questions de compétence liées au modelet adresse qui a mené à la création du fichier `ont_file`. Les réponses à ces questions sont enregistrés dans le dossier `out_folder` sous la forme des fichiers csv. Les questions sont les suivantes :
        * *Quelles sont les adresses faisant référence à un lieu dont la localisation est située dans une zone géographique donnée ?*
        * *Quelles sont les adresses faisant référence à un numéro d'immeuble situé le long d'une rue donnée ?*
        * *Quelles sont les coordonnées d'une adresse donnée (définie par son libellé) ?*

## 1. Traitements généraux

### Import des librairies

In [ ]:
import os
import urllib.parse as up
import importlib.util

### Import des variables globales

In [ ]:
ont_file = "address_ont.ttl"
data_file = "addresses.csv"
query_file = "query.txt"
mapping_file = "mapping.json"
out_ontorefine_file = "addresses-temp.ttl"
out_file = "addresses.ttl"
local_config_file = "repo_config.ttl"

out_folder = "out"
tmp_folder = "tmp"

export_format = "TURTLE"

project_name = "taito_japan"

ontorefine_url = "http://localhost:7333"
graphdb_url = "http://localhost:7200"

ontorefine_cli = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli"
ontorefine_cli = "/Applications/Ontotext\ Refine.app/Contents/app/bin/ontorefine-cli"

python_fn_file = "functions.py"
python_code_folder = "../../code"

### Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier de sortie s'il n'existe pas encore pour stocker les fichiers de sortie



In [ ]:
tmp_folder = os.path.abspath(tmp_folder)
out_folder = os.path.abspath(out_folder)
python_code_folder = os.path.abspath(python_code_folder)

ont_file = os.path.abspath(ont_file)
data_file = os.path.abspath(data_file)
query_file = os.path.abspath(query_file)
mapping_file = os.path.abspath(mapping_file)
out_ontorefine_file = os.path.abspath(os.path.join(tmp_folder,out_ontorefine_file))
out_file = os.path.join(out_folder, out_file)
python_fn_file = os.path.join(python_code_folder, python_fn_file)
local_config_file = os.path.join(tmp_folder, local_config_file)

### Import d'un module situé dans le dossier `code`

In [ ]:
# Spécifie le module qui doit être importé par rapport au chemin du fichier
spec=importlib.util.spec_from_file_location("fn",python_fn_file)

# Création d'un nouveau module basé sur spec
fn = importlib.util.module_from_spec(spec)
 
# Exécution du module dans son propre espace de noms lorsqu'un module est importé ou rechargé.
spec.loader.exec_module(fn)

### Création de dossiers s'ils n'existent pas

In [ ]:
fn.create_folder_if_not_exists(out_folder)
fn.create_folder_if_not_exists(tmp_folder)

## 2. Structuration des données contenues dans `data_file` via OntoRefine

:warning: Dans cette partie, il faut que OntoRefine soit lancé avnt d'exécuter les lignes de code.


### Création d'un projet Ontorefine à partir d'un fichier CSV défini dans `data_file`

In [ ]:
project_id = fn.create_ontorefine_project(ontorefine_cli, ontorefine_url, project_name, data_file)

### Export des données du projet en RDF à partir du fichier de mapping `mapping_file`

In [ ]:
fn.export_rdf_data_of_ontorefine_project(ontorefine_cli, ontorefine_url, project_id, mapping_file, out_ontorefine_file)

### Suppression du projet Ontorefine récemment créé

In [ ]:
fn.delete_ontorefine_project(ontorefine_cli, ontorefine_url, project_id)

## 3. Import des données dans GraphDB et nettoyage

:warning: Lancer GraphDB avant de lancer les commandes ci-dessous (pour que `graphdb_url` fonctionne).

### Création d'un répertoire dont l'id vaut `project_name`

In [ ]:
fn.create_config_local_repository_file(local_config_file, project_name)
url = f"{graphdb_url}/rest/repositories"
curl_cmd_local = fn.get_curl_command("POST", url, content_type="multipart/form-data", form=f"config=@{local_config_file}")
os.system(curl_cmd_local)

### Vidage du répertoire s'il existe déjà et qu'il n'est pas vide

In [ ]:
url = f"{graphdb_url}/repositories/{project_name}/statements"
cmd = fn.get_curl_command("DELETE", url, content_type="application/x-turtle")
os.system(cmd)

### Import de l'ontologie et des triplets créés dans la partie liée à Ontorefine

In [ ]:
url = f"{graphdb_url}/repositories/{project_name}/statements"
cmd_1 = fn.get_curl_command("POST", url, content_type="application/x-turtle", local_file=ont_file)
cmd_2 = fn.get_curl_command("POST", url, content_type="application/x-turtle", local_file=out_ontorefine_file)

os.system(cmd_1)
os.system(cmd_2)

### Nettoyage les données en supprimant les doublons

In [ ]:
# Lecture du fichier contenant la requête pour nettoyer les données
query = fn.read_file(query_file)

query_encoded = up.quote(query)
url = f"{graphdb_url}/repositories/{project_name}/statements"
cmd = fn.get_curl_command("POST", url, content_type="application/x-www-form-urlencoded", post_data=f"update={query_encoded}")

os.system(cmd)

## 4. Réponses aux questions de compétence

### Quelles sont les adresses faisant référence à un lieu dont la localisation est située dans une zone géographique donnée ?

In [ ]:
proj_uri = "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
polygon_wkt = "POLYGON ((139.780855 35.705489, 139.763281 35.704548, 139.768667 35.698118, 139.783087 35.699268, 139.780855 35.705489))"
out_file_name = "query1-out.csv"

query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX gsp: <http://www.opengis.net/ont/geosparql#>
PREFIX gspf: <http://www.opengis.net/def/function/geosparql/>

select ?s ?addressLabel ?coords where {{
    ?s a addr:Address; addr:targets [gsp:asWKT ?coords]; rdfs:label ?addressLabel.
        FILTER (
        gspf:sfWithin(
            ?coords,
            "<{proj_uri}> {polygon_wkt}"^^gsp:wktLiteral
        )
    )

}}
"""

fn.select_query(query, graphdb_url, project_name, os.path.join(out_folder,out_file_name))

### Quelles sont les adresses faisant référence à un numéro d'immeuble situées dans un district (défini par son label) ?

In [ ]:
district_name = "秋葉原"
lang = "ja"
out_file_name = "query2-out.csv"

query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
SELECT ?address ?addressLabel ?geom WHERE {{
    ?lm a addr:Landmark; addr:isLandmarkType addr:District; rdfs:label ?districtName.
    FILTER(LCASE(STR(?districtName)) = \"{district_name.lower()}\" && LANG(?districtName) = \"{lang}\")
    ?address a addr:Address; rdfs:label ?addressLabel.
    OPTIONAL {{?address addr:targets [geo:asWKT ?geom].}}
    {{?address addr:firstStep ?addrSegment}}UNION{{?address addr:firstStep [addr:nextStep+ ?addrSegment]}}
    ?addrSegment a addr:AddressSegment; addr:relatum ?lm.
}}
"""

fn.select_query(query, graphdb_url, project_name, os.path.join(out_folder,out_file_name))

### Quelles sont les coordonnées d'une adresse donnée (définie par son libellé) ?

In [ ]:
address_name = "1-11, Yaesu 秋葉原-Chome, 13106, 東京都"
lang = "ja"
out_file_name = "query3-out.csv"

query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX gsp: <http://www.opengis.net/ont/geosparql#>

SELECT ?item ?addressLabel ?coords WHERE {{
    ?item a addr:Address;
          rdfs:label ?addressLabel; 
          addr:targets [gsp:asWKT ?coords].
    FILTER(LCASE(STR(?addressLabel)) = \"{address_name.lower()}\" && LANG(?addressLabel) = \"{lang}\")
}}
"""

fn.select_query(query, graphdb_url, project_name, os.path.join(out_folder,out_file_name))